In [1]:
import pandas as pd
import pymysql
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# Configuración BD
DB_HOST = os.getenv("IP") 
DB_USER = "rrhh_master" 
DB_PASSWORD = os.getenv("clave_sql") 
DB_NAME = "rrhh_app" 

conexion = pymysql.connect(
    host=DB_HOST,
    user=DB_USER,
    password=DB_PASSWORD,
    charset='utf8mb4'
)
cursor = conexion.cursor()

In [3]:
sql_query = f"""
SELECT 
`full_name`, 
`rut`,  
`active_since`, 
`status`, 
`name_role`
FROM rrhh_app.employees
WHERE name_role IN (
'Asistente De Servicios Generales', 
'Analista de Control De Calidad', 
'Analista De Microbiología', 
'Asistente De Laboratorio', 
'Coordinadora Muestras', 
'Inspector De Proceso', 
'Administrativo Bodega Despacho', 
'Ayudante De Bodega', 
'Asistente De Bodega', 
'Chofer Administrativo Transporte', 
'Chofer', 
'Coordinador de Planta', 
'Encargado De Bodega Inflamables', 
'Encargado De Bodega Materias Primas', 
'Operario', 
'Operario Almacenamiento y Gestión de Residuos', 
'Peoneta')
AND status = 'activo'
order by active_since DESC;
"""

#1 Importar dataframes

In [4]:
from IPython.display import display
# Utiliza pandas.read_sql para ejecutar la consulta con la conexión ya abierta
df_resultado = pd.read_sql(sql_query, conexion)

print("\n--- ¡Consulta ejecutada y DataFrame creado! ---")
#display(df_resultado.head())

print(f"\nNúmero de filas y columnas: {df_resultado.shape}")
print("Columnas y tipos de datos:")
df_resultado.info()


--- ¡Consulta ejecutada y DataFrame creado! ---

Número de filas y columnas: (287, 5)
Columnas y tipos de datos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287 entries, 0 to 286
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   full_name     287 non-null    object
 1   rut           287 non-null    object
 2   active_since  287 non-null    object
 3   status        287 non-null    object
 4   name_role     287 non-null    object
dtypes: object(5)
memory usage: 11.3+ KB


C:\Users\bgacitua\AppData\Local\Temp\ipykernel_26244\846569545.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_resultado = pd.read_sql(sql_query, conexion)


In [5]:
sql_query_incidencias = f"""
SELECT DISTINCT
    ci.*, 
    e.full_name,
    e.name_role,
    e.rut,
    CASE
        WHEN ci.start_date >= '2024-12-07' AND ci.start_date <= '2025-01-06' THEN 'Enero'
        WHEN ci.start_date >= '2025-01-07' AND ci.start_date <= '2025-02-05' THEN 'Febrero'
        WHEN ci.start_date >= '2025-02-06' AND ci.start_date <= '2025-03-06' THEN 'Marzo'
        WHEN ci.start_date >= '2025-03-07' AND ci.start_date <= '2025-04-06' THEN 'Abril'
        WHEN ci.start_date >= '2025-04-07' AND ci.start_date <= '2025-05-06' THEN 'Mayo'
        WHEN ci.start_date >= '2025-05-07' AND ci.start_date <= '2025-06-05' THEN 'Junio'
        WHEN ci.start_date >= '2025-06-06' AND ci.start_date <= '2025-07-05' THEN 'Julio'
        WHEN ci.start_date >= '2025-07-06' AND ci.start_date <= '2025-08-05' THEN 'Agosto'
        WHEN ci.start_date >= '2025-08-06' AND ci.start_date <= '2025-09-04' THEN 'Septiembre'
        WHEN ci.start_date >= '2025-09-05' AND ci.start_date <= '2025-10-04' THEN 'Octubre'
        WHEN ci.start_date >= '2025-10-05' AND ci.start_date <= '2025-11-03' THEN 'Noviembre'
        WHEN ci.start_date >= '2025-11-04' AND ci.start_date <= '2025-12-03' THEN 'Diciembre'
        WHEN ci.start_date >= '2025-12-04' AND ci.start_date <= '2026-01-03' THEN 'Enero'
        ELSE 'Año anterior'
    END AS mes,
    CASE
        WHEN ci.start_date >= '2024-12-07' AND ci.start_date <= '2024-12-22' THEN '1 Quincena'
        WHEN ci.start_date >= '2024-12-23' AND ci.start_date <= '2025-01-06' THEN '2 Quincena'
        WHEN ci.start_date >= '2025-01-07' AND ci.start_date <= '2025-01-21' THEN '1 Quincena'
        WHEN ci.start_date >= '2025-01-22' AND ci.start_date <= '2025-02-05' THEN '2 Quincena'
        WHEN ci.start_date >= '2025-02-06' AND ci.start_date <= '2025-02-20' THEN '1 Quincena'
        WHEN ci.start_date >= '2025-02-21' AND ci.start_date <= '2025-03-06' THEN '2 Quincena'
        WHEN ci.start_date >= '2025-03-07' AND ci.start_date <= '2025-03-21' THEN '1 Quincena'
        WHEN ci.start_date >= '2025-03-22' AND ci.start_date <= '2025-04-06' THEN '2 Quincena'
        WHEN ci.start_date >= '2025-04-07' AND ci.start_date <= '2025-04-21' THEN '1 Quincena'
        WHEN ci.start_date >= '2025-04-22' AND ci.start_date <= '2025-05-06' THEN '2 Quincena'
        WHEN ci.start_date >= '2025-05-07' AND ci.start_date <= '2025-05-21' THEN '1 Quincena'
        WHEN ci.start_date >= '2025-05-22' AND ci.start_date <= '2025-06-05' THEN '2 Quincena'
        WHEN ci.start_date >= '2025-06-06' AND ci.start_date <= '2025-06-20' THEN '1 Quincena'
        WHEN ci.start_date >= '2025-06-21' AND ci.start_date <= '2025-07-05' THEN '2 Quincena'
        WHEN ci.start_date >= '2025-07-06' AND ci.start_date <= '2025-07-20' THEN '1 Quincena'
        WHEN ci.start_date >= '2025-07-21' AND ci.start_date <= '2025-08-05' THEN '2 Quincena'
        WHEN ci.start_date >= '2025-08-06' AND ci.start_date <= '2025-08-20' THEN '1 Quincena'
        WHEN ci.start_date >= '2025-08-21' AND ci.start_date <= '2025-09-04' THEN '2 Quincena'
        WHEN ci.start_date >= '2025-09-05' AND ci.start_date <= '2025-09-19' THEN '1 Quincena'
        WHEN ci.start_date >= '2025-09-20' AND ci.start_date <= '2025-10-04' THEN '2 Quincena'
        WHEN ci.start_date >= '2025-10-05' AND ci.start_date <= '2025-10-19' THEN '1 Quincena'
        WHEN ci.start_date >= '2025-10-20' AND ci.start_date <= '2025-11-03' THEN '2 Quincena'
        WHEN ci.start_date >= '2025-11-04' AND ci.start_date <= '2025-11-18' THEN '1 Quincena'
        WHEN ci.start_date >= '2025-11-19' AND ci.start_date <= '2025-12-03' THEN '2 Quincena'
        WHEN ci.start_date >= '2025-12-04' AND ci.start_date <= '2025-12-18' THEN '1 Quincena'
        WHEN ci.start_date >= '2025-12-19' AND ci.start_date <= '2026-01-03' THEN '2 Quincena'
        ELSE 'Año anterior'
    END AS quincena
FROM 
    rrhh_app.consolidado_incidencias ci
INNER JOIN 
    rrhh_app.employees e ON ci.employee_id = e.id
WHERE ci.start_date > '2025-08-05' and ci.start_date < '2025-09-04'
AND e.name_role IN (
'Asistente De Servicios Generales', 
'Analista de Control De Calidad', 
'Analista De Microbiología', 
'Asistente De Laboratorio', 
'Coordinadora Muestras', 
'Inspector De Proceso', 
'Administrativo Bodega Despacho', 
'Ayudante De Bodega', 
'Asistente De Bodega', 
'Chofer Administrativo Transporte', 
'Chofer', 
'Coordinador de Planta', 
'Encargado De Bodega Inflamables', 
'Encargado De Bodega Materias Primas', 
'Operario', 
'Operario Almacenamiento y Gestión de Residuos', 
'Peoneta');
"""

In [6]:
# Utiliza pandas.read_sql para ejecutar la consulta con la conexión ya abierta
df_incidencias = pd.read_sql(sql_query_incidencias, conexion)

print("\n--- ¡Consulta ejecutada y DataFrame creado! ---")
#display(df_resultado.head())

print(f"\nNúmero de filas y columnas: {df_incidencias.shape}")
print("Columnas y tipos de datos:")
df_incidencias.info()
display(df_incidencias)


--- ¡Consulta ejecutada y DataFrame creado! ---

Número de filas y columnas: (26, 16)
Columnas y tipos de datos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    26 non-null     object
 1   start_date            26 non-null     object
 2   end_date              26 non-null     object
 3   days_count            26 non-null     object
 4   workday_stage         26 non-null     object
 5   application_date      26 non-null     object
 6   application_end_date  26 non-null     object
 7   employee_id           26 non-null     object
 8   status                26 non-null     object
 9   created_at            26 non-null     object
 10  tabla_origen          26 non-null     object
 11  full_name             26 non-null     object
 12  name_role             26 non-null     object
 13  rut                   26 non

C:\Users\bgacitua\AppData\Local\Temp\ipykernel_26244\1293398792.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_incidencias = pd.read_sql(sql_query_incidencias, conexion)


,id,start_date,end_date,days_count,workday_stage,application_date,application_end_date,employee_id,status,created_at,tabla_origen,full_name,name_role,rut,mes,quincena
0,108847,2025-08-25,2025-08-26,2,full_working_day,2025-08-25,2025-08-26,4319,approved,2025-08-27T10:17:04.539-04:00,permissions,Cesar Antonio Solis Cartes,Asistente De Laboratorio,15.172.551-1,Septiembre,2 Quincena
1,108880,2025-08-25,2025-08-25,1,full_working_day,2025-08-25,2025-08-25,4861,approved,2025-08-27T11:27:13.759-04:00,permissions,Juan Manuel González Soto,Asistente De Laboratorio,20.580.807-8,Septiembre,2 Quincena
2,108946,2025-08-27,2025-08-27,1,start_working_day,2025-08-27,2025-08-27,4319,approved,2025-08-28T08:12:22.638-04:00,permissions,Cesar Antonio Solis Cartes,Asistente De Laboratorio,15.172.551-1,Septiembre,2 Quincena
3,109045,2025-09-01,2025-09-01,1,full_working_day,2025-09-01,2025-09-01,6666,requested,2025-08-31T20:56:03.247-04:00,permissions,Nicolás Alexis Sandoval Contreras,Ayudante De Bodega,20.580.404-8,Septiembre,2 Quincena
4,109078,2025-09-02,2025-09-02,0.18292683,full_working_day,2025-09-02,2025-09-02,11587,approved,2025-09-01T16:58:54.932-04:00,permissions,Marcelo Del Carmen González Baeza,Operario,14.485.044-0,Septiembre,2 Quincena
5,109244,2025-08-29,2025-08-29,1,full_working_day,2025-08-29,2025-08-29,4304,approved,2025-09-03T11:19:17.252-04:00,permissions,Joel Esteban Roman Gonzalez,Operario,13.935.403-6,Septiembre,2 Quincena
6,109309,2025-09-01,2025-09-01,1,full_working_day,2025-09-01,2025-09-01,8153,approved,2025-09-03T15:50:52.998-04:00,permissions,Álvaro José Astorga Hinostroza,Operario,15.820.779-6,Septiembre,2 Quincena
7,108286,2025-08-08,2025-08-08,1,full_working_day,2025-08-08,2025-08-08,7227,approved,2025-08-14T08:48:04.315-04:00,absences,Diego Alejandro Islas Huenchuman,Ayudante De Bodega,18.765.417-3,Septiembre,1 Quincena
8,108385,2025-08-13,2025-08-13,1,full_working_day,2025-08-13,2025-08-13,8811,approved,2025-08-18T10:06:31.762-04:00,absences,Matías Sebastián Muñoz Lagos,Operario,17.781.202-1,Septiembre,1 Quincena
9,108980,2025-08-25,2025-08-25,1,full_working_day,2025-09-01,2025-09-01,10956,approved,2025-08-28T13:20:54.832-04:00,absences,Paz Belén Fuentes Melo,Asistente De Laboratorio,21.289.225-4,Septiembre,2 Quincena


In [7]:
#Importa dataframe de marcas corregidas
df_marcas = pd.read_excel(r"C:\Users\bgacitua\Desktop\Repositorio_GitHub\Scripts de Python\Script Bono Asistencia\Archivos de trabajo\reporte_turnos_marcas_corregidas_para_base_septiembre_bono.xlsx")
#display(df_marcas.head())
df_marcas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1013 entries, 0 to 1012
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Rut                          1013 non-null   object
 1   Nombre                       1013 non-null   object
 2   Trabajo - Cargo              1013 non-null   object
 3   Unidad                       1013 non-null   object
 4   Puesto                       1013 non-null   object
 5   Fecha                        1013 non-null   object
 6   Periodo                      1013 non-null   object
 7   Tipo de turno                1013 non-null   object
 8   Entrada programada           1013 non-null   object
 9   Salida programada            1013 non-null   object
 10  Marca de entrada             1013 non-null   object
 11  Marca de salida              1013 non-null   object
 12  Motivo marca de entrada      556 non-null    object
 13  Comentario marca de entrada  530 

In [8]:
#Importar dataframe atrasos
columnas_necesarias = ['nombreTipoAusencia' ,'fecha', 'horaInicio', 'horaProgramadaInicio', 'HoraInicioMarcaReloj', 'horaTermino', 'horaProgramadaTermino', 'HoraTerminoMarcaReloj', 'rut', 'nombre', 'apellidoPaterno', 'apellidoMaterno', 'Centrocosto_trabajador', 'PuestoIntegracion_trabajador', 'nombreMotivoPago']
df_atrasos = pd.read_excel(r"C:\Users\bgacitua\Desktop\Repositorio_GitHub\Scripts de Python\Script Bono Asistencia\Archivos de trabajo\2025-09-09 09_40_17-715030462-reporte_turnos_bruto_combinado.xlsx")
df_atrasos = df_atrasos[columnas_necesarias]
df_atrasos.info()

# -- Crear dataframe con las incidencias de los empleados. Puede ser que haya discrepancias con BUK -- #


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10750 entries, 0 to 10749
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   nombreTipoAusencia            10750 non-null  object
 1   fecha                         10750 non-null  object
 2   horaInicio                    10750 non-null  object
 3   horaProgramadaInicio          10750 non-null  object
 4   HoraInicioMarcaReloj          10750 non-null  object
 5   horaTermino                   10750 non-null  object
 6   horaProgramadaTermino         10750 non-null  object
 7   HoraTerminoMarcaReloj         10750 non-null  object
 8   rut                           10750 non-null  object
 9   nombre                        10750 non-null  object
 10  apellidoPaterno               10750 non-null  object
 11  apellidoMaterno               10750 non-null  object
 12  Centrocosto_trabajador        10750 non-null  object
 13  PuestoIntegracio

In [9]:
display(df_atrasos.head())

,nombreTipoAusencia,fecha,horaInicio,horaProgramadaInicio,HoraInicioMarcaReloj,horaTermino,horaProgramadaTermino,HoraTerminoMarcaReloj,rut,nombre,apellidoPaterno,apellidoMaterno,Centrocosto_trabajador,PuestoIntegracion_trabajador,nombreMotivoPago
0,,2025-08-06,07:50:00,2025-08-06 07:50:00,2025-08-06 07:50:00,17:00:00,2025-08-06 17:00:00,2025-08-06 17:00:00,69866409,Gloria,Gonzalez,Castro,Administración,Secretaria,
1,,2025-08-07,07:50:00,2025-08-07 07:50:00,2025-08-07 07:02:01,17:00:00,2025-08-07 17:00:00,2025-08-07 16:57:16,69866409,Gloria,Gonzalez,Castro,Administración,Secretaria,
2,,2025-08-08,07:50:00,2025-08-08 07:50:00,2025-08-08 07:30:44,15:30:00,2025-08-08 15:30:00,2025-08-08 15:30:00,69866409,Gloria,Gonzalez,Castro,Administración,Secretaria,
3,,2025-08-11,07:50:00,2025-08-11 07:50:00,2025-08-11 07:23:45,17:00:00,2025-08-11 17:00:00,2025-08-11 17:01:52,69866409,Gloria,Gonzalez,Castro,Administración,Secretaria,
4,,2025-08-12,07:50:00,2025-08-12 07:50:00,2025-08-12 07:50:00,17:00:00,2025-08-12 17:00:00,2025-08-12 16:55:48,69866409,Gloria,Gonzalez,Castro,Administración,Secretaria,


#2 Exploración y limpieza de dataframes

In [10]:
df_marcas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1013 entries, 0 to 1012
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Rut                          1013 non-null   object
 1   Nombre                       1013 non-null   object
 2   Trabajo - Cargo              1013 non-null   object
 3   Unidad                       1013 non-null   object
 4   Puesto                       1013 non-null   object
 5   Fecha                        1013 non-null   object
 6   Periodo                      1013 non-null   object
 7   Tipo de turno                1013 non-null   object
 8   Entrada programada           1013 non-null   object
 9   Salida programada            1013 non-null   object
 10  Marca de entrada             1013 non-null   object
 11  Marca de salida              1013 non-null   object
 12  Motivo marca de entrada      556 non-null    object
 13  Comentario marca de entrada  530 

In [11]:
display(df_marcas.head())
print(f"Número de filas: {df_marcas.shape[0]}")

,Rut,Nombre,Trabajo - Cargo,Unidad,Puesto,Fecha,Periodo,Tipo de turno,Entrada programada,Salida programada,Marca de entrada,Marca de salida,Motivo marca de entrada,Comentario marca de entrada,Motivo marca de salida,Comentario marca de salida,Tipo de corrección
0,6986640-9,Gloria Emilia Gonzalez Castro,Secretaria,Administación y Lavandería,Secretaria,06-08-2025,1 Quincena,T,07:50,17:00,07:50:00 *,17:00:00 *,Olvido de Marcaje,NaN,Olvido de Marcaje,NaN,Entrada y salida
1,6986640-9,Gloria Emilia Gonzalez Castro,Secretaria,Administación y Lavandería,Secretaria,08-08-2025,1 Quincena,D,07:50,15:30,07:30:44,15:30:00 *,NaN,NaN,Olvido de Marcaje,Olvido marca,Solo salida
2,6986640-9,Gloria Emilia Gonzalez Castro,Secretaria,Administación y Lavandería,Secretaria,12-08-2025,1 Quincena,T,07:50,17:00,07:50:00 *,16:55:48,Olvido de Marcaje,NaN,NaN,NaN,Solo entrada
3,6986640-9,Gloria Emilia Gonzalez Castro,Secretaria,Administación y Lavandería,Secretaria,28-08-2025,2 Quincena,T,07:50,17:00,07:03:12,17:00:00 *,NaN,NaN,Olvido de Marcaje,Olvido marca,Solo salida
4,19243690-7,María Francisca Ulloa Valdés,Analista De Innovación,Desarrollo Sabores General,Analista,06-08-2025,1 Quincena,T,07:50,17:00,07:36:28,17:21:00,NaN,NaN,NaN,NaN,Entrada y salida


Número de filas: 1013


In [12]:
#Eliminar filas con 'Falla Reloj' y 'Otro' en ambas columnas
df_marcas_filtrado = df_marcas[
    ~df_marcas['Motivo marca de entrada'].isin(['Falla Reloj', 'Otro']) & 
    ~df_marcas['Motivo marca de salida'].isin(['Falla Reloj', 'Otro'])
]

print(f"Filas originales: {df_marcas.shape[0]}")
print(f"Filas después del filtro: {df_marcas_filtrado.shape[0]}")
display(df_marcas_filtrado.head())

Filas originales: 1013
Filas después del filtro: 634


,Rut,Nombre,Trabajo - Cargo,Unidad,Puesto,Fecha,Periodo,Tipo de turno,Entrada programada,Salida programada,Marca de entrada,Marca de salida,Motivo marca de entrada,Comentario marca de entrada,Motivo marca de salida,Comentario marca de salida,Tipo de corrección
0,6986640-9,Gloria Emilia Gonzalez Castro,Secretaria,Administación y Lavandería,Secretaria,06-08-2025,1 Quincena,T,07:50,17:00,07:50:00 *,17:00:00 *,Olvido de Marcaje,NaN,Olvido de Marcaje,NaN,Entrada y salida
1,6986640-9,Gloria Emilia Gonzalez Castro,Secretaria,Administación y Lavandería,Secretaria,08-08-2025,1 Quincena,D,07:50,15:30,07:30:44,15:30:00 *,NaN,NaN,Olvido de Marcaje,Olvido marca,Solo salida
2,6986640-9,Gloria Emilia Gonzalez Castro,Secretaria,Administación y Lavandería,Secretaria,12-08-2025,1 Quincena,T,07:50,17:00,07:50:00 *,16:55:48,Olvido de Marcaje,NaN,NaN,NaN,Solo entrada
3,6986640-9,Gloria Emilia Gonzalez Castro,Secretaria,Administación y Lavandería,Secretaria,28-08-2025,2 Quincena,T,07:50,17:00,07:03:12,17:00:00 *,NaN,NaN,Olvido de Marcaje,Olvido marca,Solo salida
4,19243690-7,María Francisca Ulloa Valdés,Analista De Innovación,Desarrollo Sabores General,Analista,06-08-2025,1 Quincena,T,07:50,17:00,07:36:28,17:21:00,NaN,NaN,NaN,NaN,Entrada y salida


In [13]:
## -- Limpiar df_atrasos -- ##

#Limpieza del formato de las columnas con datetime
columnas_a_limpiar = ['horaProgramadaInicio', 'HoraInicioMarcaReloj', 'horaProgramadaTermino', 'HoraTerminoMarcaReloj']

for columna in columnas_a_limpiar:
    if columna in df_atrasos.columns:
        # Dividir por espacio y tomar la parte después del primer espacio
        df_atrasos[columna] = df_atrasos[columna].str.split(' ', n=1, expand=True)[1]

df_atrasos['nombreCompleto'] = df_atrasos['nombre'].str.cat([
    df_atrasos['apellidoPaterno'], 
    df_atrasos['apellidoMaterno']], sep=' ', na_rep='')

#Verificar los cambios
display(df_atrasos.head())

#Filtrar por los puestos sujetos a bono asistencia
puestos_filtrados = [
'Asistente De Servicios Generales', 
'Analista de Control De Calidad', 
'Analista De Microbiología', 
'Asistente De Laboratorio', 
'Coordinadora Muestras', 
'Inspector De Proceso', 
'Administrativo Bodega Despacho', 
'Ayudante De Bodega', 
'Asistente De Bodega', 
'Chofer Administrativo Transporte', 
'Chofer', 
'Coordinador de Planta', 
'Encargado De Bodega Inflamables', 
'Encargado De Bodega Materias Primas', 
'Operario', 
'Operario Almacenamiento y Gestión de Residuos', 
'Peoneta']

df_atrasos_filtrado = df_atrasos[df_atrasos['PuestoIntegracion_trabajador'].isin(puestos_filtrados)]
print(f"Filas originales: {df_atrasos.shape[0]}")
print(f"Filas después del filtro: {df_atrasos_filtrado.shape[0]}")
display(df_atrasos_filtrado.head())

,nombreTipoAusencia,fecha,horaInicio,horaProgramadaInicio,HoraInicioMarcaReloj,horaTermino,horaProgramadaTermino,HoraTerminoMarcaReloj,rut,nombre,apellidoPaterno,apellidoMaterno,Centrocosto_trabajador,PuestoIntegracion_trabajador,nombreMotivoPago,nombreCompleto
0,,2025-08-06,07:50:00,07:50:00,07:50:00,17:00:00,17:00:00,17:00:00,69866409,Gloria,Gonzalez,Castro,Administración,Secretaria,,Gloria Gonzalez Castro
1,,2025-08-07,07:50:00,07:50:00,07:02:01,17:00:00,17:00:00,16:57:16,69866409,Gloria,Gonzalez,Castro,Administración,Secretaria,,Gloria Gonzalez Castro
2,,2025-08-08,07:50:00,07:50:00,07:30:44,15:30:00,15:30:00,15:30:00,69866409,Gloria,Gonzalez,Castro,Administración,Secretaria,,Gloria Gonzalez Castro
3,,2025-08-11,07:50:00,07:50:00,07:23:45,17:00:00,17:00:00,17:01:52,69866409,Gloria,Gonzalez,Castro,Administración,Secretaria,,Gloria Gonzalez Castro
4,,2025-08-12,07:50:00,07:50:00,07:50:00,17:00:00,17:00:00,16:55:48,69866409,Gloria,Gonzalez,Castro,Administración,Secretaria,,Gloria Gonzalez Castro


Filas originales: 10750
Filas después del filtro: 5664


,nombreTipoAusencia,fecha,horaInicio,horaProgramadaInicio,HoraInicioMarcaReloj,horaTermino,horaProgramadaTermino,HoraTerminoMarcaReloj,rut,nombre,apellidoPaterno,apellidoMaterno,Centrocosto_trabajador,PuestoIntegracion_trabajador,nombreMotivoPago,nombreCompleto
7,,2025-08-06,07:50:00,07:50:00,07:20:36,17:00:00,17:00:00,16:55:11,128294988,Cesar,Vera,Ibarra,Administración,Asistente De Servicios Generales,,Cesar Vera Ibarra
8,,2025-08-07,07:50:00,07:50:00,07:18:57,17:00:00,17:00:00,16:52:30,128294988,Cesar,Vera,Ibarra,Administración,Asistente De Servicios Generales,,Cesar Vera Ibarra
9,,2025-08-08,07:50:00,07:50:00,07:27:16,15:30:00,15:30:00,15:30:00,128294988,Cesar,Vera,Ibarra,Administración,Asistente De Servicios Generales,,Cesar Vera Ibarra
10,,2025-08-11,07:50:00,07:50:00,07:19:18,17:00:00,17:00:00,16:53:47,128294988,Cesar,Vera,Ibarra,Administración,Asistente De Servicios Generales,,Cesar Vera Ibarra
11,,2025-08-12,07:50:00,07:50:00,07:15:15,17:00:00,17:00:00,16:53:18,128294988,Cesar,Vera,Ibarra,Administración,Asistente De Servicios Generales,,Cesar Vera Ibarra


In [14]:
#Info() de los dataframes finales
df_resultado.info()
df_marcas_filtrado.info()
df_atrasos_filtrado.info()
df_incidencias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287 entries, 0 to 286
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   full_name     287 non-null    object
 1   rut           287 non-null    object
 2   active_since  287 non-null    object
 3   status        287 non-null    object
 4   name_role     287 non-null    object
dtypes: object(5)
memory usage: 11.3+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 634 entries, 0 to 1012
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Rut                          634 non-null    object
 1   Nombre                       634 non-null    object
 2   Trabajo - Cargo              634 non-null    object
 3   Unidad                       634 non-null    object
 4   Puesto                       634 non-null    object
 5   Fecha                        634 non-null    object
 6  

#3 Procesamiento de dataframes

In [15]:
#Agrupamiento y resumen de permisos, licencias e inasistencias

resumen_incidencias = df_incidencias.groupby(['rut', 'full_name', 'tabla_origen']).agg({
    'tabla_origen': 'count',  # Contar total de incidencias
    'start_date': ['min', 'max']  # Fecha primera y última incidencia
}).reset_index()

# Aplanar las columnas
resumen_incidencias.columns = ['rut', 'full_name', 'tabla_origen', 'total_incidencias', 'primera_incidencia', 'ultima_incidencia']

print(f'Cantidad de personas encontrados = {resumen_incidencias.shape[0]}')
display(resumen_incidencias)

Cantidad de personas encontrados = 23


,rut,full_name,tabla_origen,total_incidencias,primera_incidencia,ultima_incidencia
0,12.314.132-6,Richard Agustin Bustamante Castro,licences,1,2025-09-02,2025-09-02
1,13.558.836-9,Marcelo Nicanor Morales Barrera,licences,1,2025-08-11,2025-08-11
2,13.935.403-6,Joel Esteban Roman Gonzalez,permissions,1,2025-08-29,2025-08-29
3,14.485.044-0,Marcelo Del Carmen González Baeza,permissions,1,2025-09-02,2025-09-02
4,15.172.551-1,Cesar Antonio Solis Cartes,permissions,2,2025-08-25,2025-08-27
5,15.820.779-6,Álvaro José Astorga Hinostroza,absences,1,2025-09-02,2025-09-02
6,15.820.779-6,Álvaro José Astorga Hinostroza,permissions,1,2025-09-01,2025-09-01
7,15.934.416-9,Alexander Araos Deramond,absences,1,2025-08-29,2025-08-29
8,16.323.864-0,Francisco Javier Vásquez Adrián,licences,1,2025-08-23,2025-08-23
9,17.049.574-8,Patricio Andrés Gutiérrez Pérez,licences,1,2025-08-10,2025-08-10


In [16]:
#Columnas df_atrasos_filtrado
'''
columnas_df_atrasos_filt = df_atrasos_filtrado.columns.tolist()
print(columnas_df_atrasos_filt)

tipos_ausencias = df_atrasos_filtrado['nombreTipoAusencia'].unique()
print(tipos_ausencias)
'''
#Eliminar filas vacías
df_atrasos_filtrado_permisos = df_atrasos_filtrado.copy()

# Eliminar filas donde 'nombreTipoAusencia' es None, NaN, vacío o solo espacios
df_atrasos_filtrado_permisos = df_atrasos_filtrado_permisos[
    df_atrasos_filtrado_permisos['nombreTipoAusencia'].notna() &
    (df_atrasos_filtrado_permisos['nombreTipoAusencia'].str.strip() != '')
]

print(df_atrasos_filtrado_permisos['nombreTipoAusencia'].unique())

['Vacación' 'Licencia postnatal' 'Licencia Accidente Trayecto'
 'Permiso sin goce de sueldo' 'Permiso con goce de sueldo'
 'Vacación Progresiva' 'Inasistencia' 'Licencia' 'Permiso por matrimonio'
 'Licencia Accidente Trabajo' 'Licencia Accidente' 'Capacitacion']


In [17]:
#Insertar datos encontrados en Reporte Rflex Bruto Combinado
resumen_atrasos_filtrado_permisos = df_atrasos_filtrado_permisos.groupby(['nombreTipoAusencia', 'nombreCompleto']).agg({
    'nombreCompleto': 'count',
    'fecha': ['min', 'max']
})

import pandas as pd

# Mostrar hasta 100 filas (puedes ajustar el número si necesitas más)
#pd.set_option('display.max_rows', 100)

display(resumen_atrasos_filtrado_permisos.head())

nombreCompleto       fecha  \
                                                     count         min   
nombreTipoAusencia nombreCompleto                                        
Capacitacion       Cesar Solis Cartes                    2  2025-08-25   
Inasistencia       Alexander Araos Deramond              1  2025-08-29   
                   Diego Islas Huenchuman                1  2025-08-08   
                   Matías Muñoz Lagos                    2  2025-08-13   
                   Paz Fuentes Melo                      1  2025-08-25   

                                                         
                                                    max  
nombreTipoAusencia nombreCompleto                        
Capacitacion       Cesar Solis Cartes        2025-08-26  
Inasistencia       Alexander Araos Deramond  2025-08-29  
                   Diego Islas Huenchuman    2025-08-08  
                   Matías Muñoz Lagos        2025-08-28  
                   Paz Fuentes Melo          2025-08-25

In [18]:
print(df_atrasos_filtrado_permisos['nombreTipoAusencia'].unique())

['Vacación' 'Licencia postnatal' 'Licencia Accidente Trayecto'
 'Permiso sin goce de sueldo' 'Permiso con goce de sueldo'
 'Vacación Progresiva' 'Inasistencia' 'Licencia' 'Permiso por matrimonio'
 'Licencia Accidente Trabajo' 'Licencia Accidente' 'Capacitacion']


In [19]:
#Olvido marcas -- Procesar reporte de marcas corregidas y reporte bruto combinado de rflex. 
#Filtrar y agrupar a las personas con olvido de marcas

#Reporte de marcas corregidas - Rflex
df_marcas_filtrado = df_marcas_filtrado[df_marcas_filtrado['Trabajo - Cargo'].isin(puestos_filtrados)]

resumen_olvido_marcas = df_marcas_filtrado.groupby(['Rut', 'Nombre']).agg({
    'Motivo marca de entrada': 'count',
    'Motivo marca de salida' : 'count'
})  # Contar total de olvido de marcas

print("Olvido de marcas:")
print(f'Cantidad de personas encontrados = {resumen_olvido_marcas.shape[0]}')
#display(resumen_olvido_marcas)
display(resumen_olvido_marcas[resumen_olvido_marcas['Motivo marca de entrada'] != 0])

Olvido de marcas:
Cantidad de personas encontrados = 75


,,Motivo marca de entrada,Motivo marca de salida
Rut,Nombre,,
10706013-8,Francisco Alejandro Soto Nuñez,2,0
10875002-2,Ricardo Alejandro Muñoz Escobar,1,2
12474911-5,Marcelo Ricardo Salazar Alvarez,2,0
12632762-5,Juan Francisco Torres Olguin,1,0
12634159-8,Rodrigo David Arenas Guerra,1,0
12857871-4,Sixto Manuel Beiza Mardones,1,0
13697731-8,Ricardo Antonio Silva Tapia,2,0
14194403-7,Manuel Jesus Abarca Moreira,1,0
15464491-1,Juan Pablo Sanchez Montano,1,0


In [20]:
# Seleccionar columnas que contienen 'hora' en su nombre (ignorando mayúsculas/minúsculas)
columna_horas = df_atrasos_filtrado.columns[df_atrasos_filtrado.columns.str.contains('hora', case=False, na=False)]
df_atrasos_filtrado['nombreTipoAusencia'] = df_atrasos_filtrado['nombreTipoAusencia'].str.replace(' ', 'Sin permiso')

df_sin_permiso = df_atrasos_filtrado[df_atrasos_filtrado['nombreTipoAusencia'] == 'Sin permiso']

display(df_sin_permiso[['nombreTipoAusencia', 'nombreCompleto', 'fecha'] + columna_horas.tolist()])

C:\Users\bgacitua\AppData\Local\Temp\ipykernel_26244\2454795338.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_atrasos_filtrado['nombreTipoAusencia'] = df_atrasos_filtrado['nombreTipoAusencia'].str.replace(' ', 'Sin permiso')


,nombreTipoAusencia,nombreCompleto,fecha,horaInicio,horaProgramadaInicio,HoraInicioMarcaReloj,horaTermino,horaProgramadaTermino,HoraTerminoMarcaReloj
7,Sin permiso,Cesar Vera Ibarra,2025-08-06,07:50:00,07:50:00,07:20:36,17:00:00,17:00:00,16:55:11
8,Sin permiso,Cesar Vera Ibarra,2025-08-07,07:50:00,07:50:00,07:18:57,17:00:00,17:00:00,16:52:30
9,Sin permiso,Cesar Vera Ibarra,2025-08-08,07:50:00,07:50:00,07:27:16,15:30:00,15:30:00,15:30:00
10,Sin permiso,Cesar Vera Ibarra,2025-08-11,07:50:00,07:50:00,07:19:18,17:00:00,17:00:00,16:53:47
11,Sin permiso,Cesar Vera Ibarra,2025-08-12,07:50:00,07:50:00,07:15:15,17:00:00,17:00:00,16:53:18
...,...,...,...,...,...,...,...,...,...
10741,Sin permiso,Benjamín Cornejo Cárdenas,2025-08-29,07:50:00,07:50:00,08:01:25,14:30:00,14:30:00,14:29:31
10742,Sin permiso,Benjamín Cornejo Cárdenas,2025-09-01,07:50:00,07:50:00,07:51:23,17:00:00,17:00:00,17:00:46
10743,Sin permiso,Benjamín Cornejo Cárdenas,2025-09-02,07:50:00,07:50:00,08:00:48,17:00:00,17:00:00,17:00:50
10744,Sin permiso,Benjamín Cornejo Cárdenas,2025-09-03,07:50:00,07:50:00,07:58:10,17:00:00,17:00:00,17:00:57


In [21]:

df_sin_permiso['HoraInicioMarcaReloj'] = df_sin_permiso['HoraInicioMarcaReloj'].str.replace(' ', 'Omisión de marca')
df_sin_permiso['HoraTerminoMarcaReloj'] = df_sin_permiso['HoraTerminoMarcaReloj'].str.replace(' ', 'Omisión de marca')

C:\Users\bgacitua\AppData\Local\Temp\ipykernel_26244\3989723926.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sin_permiso['HoraInicioMarcaReloj'] = df_sin_permiso['HoraInicioMarcaReloj'].str.replace(' ', 'Omisión de marca')
C:\Users\bgacitua\AppData\Local\Temp\ipykernel_26244\3989723926.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sin_permiso['HoraTerminoMarcaReloj'] = df_sin_permiso['HoraTerminoMarcaReloj'].str.replace(' ', 'Omisión de marca')


In [22]:
df_omisiones = df_sin_permiso[(df_sin_permiso['HoraInicioMarcaReloj'] == 'Omisión de marca') | (df_sin_permiso['HoraTerminoMarcaReloj'] == 'Omisión de marca')]
display(df_omisiones)

#No hay omisiones de marca en el periodo por rflex. Lo que significa que todas fueron resultas en sistema. 

,nombreTipoAusencia,fecha,horaInicio,horaProgramadaInicio,HoraInicioMarcaReloj,horaTermino,horaProgramadaTermino,HoraTerminoMarcaReloj,rut,nombre,apellidoPaterno,apellidoMaterno,Centrocosto_trabajador,PuestoIntegracion_trabajador,nombreMotivoPago,nombreCompleto


In [23]:
#Procesar atrasos desde reporte bruto combinado de rflex
import pandas as pd
columnas_necesarias = ['nombreTipoAusencia' ,'fecha', 'horaInicio', 'horaProgramadaInicio', 'HoraInicioMarcaReloj', 'horaTermino', 'horaProgramadaTermino', 'HoraTerminoMarcaReloj', 'rut', 'nombre', 'apellidoPaterno', 'apellidoMaterno', 'Centrocosto_trabajador', 'PuestoIntegracion_trabajador', 'nombreMotivoPago']
df_atrasos_calculo = pd.read_excel(r"C:\Users\bgacitua\Desktop\Repositorio_GitHub\Scripts de Python\Script Bono Asistencia\Archivos de trabajo\2025-09-09 09_40_17-715030462-reporte_turnos_bruto_combinado.xlsx")

df_atrasos_calculo = df_atrasos_calculo[columnas_necesarias]
display(df_atrasos_calculo.head())

,nombreTipoAusencia,fecha,horaInicio,horaProgramadaInicio,HoraInicioMarcaReloj,horaTermino,horaProgramadaTermino,HoraTerminoMarcaReloj,rut,nombre,apellidoPaterno,apellidoMaterno,Centrocosto_trabajador,PuestoIntegracion_trabajador,nombreMotivoPago
0,,2025-08-06,07:50:00,2025-08-06 07:50:00,2025-08-06 07:50:00,17:00:00,2025-08-06 17:00:00,2025-08-06 17:00:00,69866409,Gloria,Gonzalez,Castro,Administración,Secretaria,
1,,2025-08-07,07:50:00,2025-08-07 07:50:00,2025-08-07 07:02:01,17:00:00,2025-08-07 17:00:00,2025-08-07 16:57:16,69866409,Gloria,Gonzalez,Castro,Administración,Secretaria,
2,,2025-08-08,07:50:00,2025-08-08 07:50:00,2025-08-08 07:30:44,15:30:00,2025-08-08 15:30:00,2025-08-08 15:30:00,69866409,Gloria,Gonzalez,Castro,Administración,Secretaria,
3,,2025-08-11,07:50:00,2025-08-11 07:50:00,2025-08-11 07:23:45,17:00:00,2025-08-11 17:00:00,2025-08-11 17:01:52,69866409,Gloria,Gonzalez,Castro,Administración,Secretaria,
4,,2025-08-12,07:50:00,2025-08-12 07:50:00,2025-08-12 07:50:00,17:00:00,2025-08-12 17:00:00,2025-08-12 16:55:48,69866409,Gloria,Gonzalez,Castro,Administración,Secretaria,


In [24]:
# Convertir las columnas a formato datetime, ignorando errores y valores vacíos
df_atrasos_calculo['horaProgramadaInicio'] = pd.to_datetime(df_atrasos_calculo['horaProgramadaInicio'], errors='coerce')
df_atrasos_calculo['HoraInicioMarcaReloj'] = pd.to_datetime(df_atrasos_calculo['HoraInicioMarcaReloj'], errors='coerce')
df_atrasos_calculo['horaProgramadaTermino'] = pd.to_datetime(df_atrasos_calculo['horaProgramadaTermino'], errors='coerce')
df_atrasos_calculo['HoraTerminoMarcaReloj'] = pd.to_datetime(df_atrasos_calculo['HoraTerminoMarcaReloj'], errors='coerce')

# Calcular la diferencia de tiempo para cada fila
df_atrasos_calculo['Diferencia'] = df_atrasos_calculo['horaProgramadaInicio'] - df_atrasos_calculo['HoraInicioMarcaReloj']

# Extraer los minutos y segundos de la columna 'Diferencia'
df_atrasos_calculo['minutos_antes'] = df_atrasos_calculo['Diferencia'].dt.total_seconds() / 60
df_atrasos_calculo['segundos_antes'] = df_atrasos_calculo['Diferencia'].dt.total_seconds() % 60

In [28]:
df_atrasos_calculo['nombreCompleto'] = df_atrasos_calculo['nombre'].str.cat([
    df_atrasos_calculo['apellidoPaterno'], 
    df_atrasos_calculo['apellidoMaterno']], sep=' ', na_rep='')

df_atrasos_calculo = df_atrasos_calculo[df_atrasos_calculo['PuestoIntegracion_trabajador'].isin(puestos_filtrados)]
print(f"Filas originales: {df_atrasos_calculo.shape[0]}")
print(f"Filas después del filtro: {df_atrasos_calculo.shape[0]}")

horas_filtradas = df_atrasos_calculo[['nombreCompleto', 'PuestoIntegracion_trabajador', 'fecha', 'horaProgramadaInicio', 'HoraInicioMarcaReloj', 'minutos_antes', 'nombreTipoAusencia']]
permisos_aux = [' ']
horas_permisos_filtrados = horas_filtradas[horas_filtradas['nombreTipoAusencia'].isin(permisos_aux)]

horas_permisos_exportar = horas_permisos_filtrados[horas_permisos_filtrados['minutos_antes'] < -10]

agrupado_permisos_exportar = horas_permisos_exportar.groupby(['nombreCompleto', 'PuestoIntegracion_trabajador']).agg({
    'minutos_antes': 'count',
    'fecha': ['min', 'max']
}).reset_index()

display(horas_permisos_exportar)
display(agrupado_permisos_exportar)

#ruta = r"C:\Users\bgacitua\Desktop\Respaldo marcas Morpho\horas_permisos_exportar.xlsx"
#horas_permisos_exportar.to_excel(ruta, index=False)





Filas originales: 5664
Filas después del filtro: 5664


,nombreCompleto,PuestoIntegracion_trabajador,fecha,horaProgramadaInicio,HoraInicioMarcaReloj,minutos_antes,nombreTipoAusencia
160,Marie Etienne,Asistente De Servicios Generales,2025-09-02,2025-09-02 11:00:00,2025-09-02 11:11:04,-11.066667,
251,Eyver Roa Vargas,Operario,2025-08-11,2025-08-11 07:50:00,2025-08-11 08:01:05,-11.083333,
262,Eyver Roa Vargas,Operario,2025-08-27,2025-08-27 07:50:00,2025-08-27 08:18:06,-28.100000,
264,Eyver Roa Vargas,Operario,2025-08-29,2025-08-29 07:50:00,2025-08-29 08:08:25,-18.416667,
991,Bastian Santander Cruces,Inspector De Proceso,2025-08-19,2025-08-19 07:50:00,2025-08-19 10:42:03,-172.050000,
...,...,...,...,...,...,...,...
10693,Juan González Soto,Asistente De Laboratorio,2025-08-27,2025-08-27 07:50:00,2025-08-27 08:15:07,-25.116667,
10699,Juan González Soto,Asistente De Laboratorio,2025-09-04,2025-09-04 07:50:00,2025-09-04 08:07:20,-17.333333,
10728,Benjamín Cornejo Cárdenas,Asistente De Laboratorio,2025-08-11,2025-08-11 07:50:00,2025-08-11 11:11:15,-201.250000,
10741,Benjamín Cornejo Cárdenas,Asistente De Laboratorio,2025-08-29,2025-08-29 07:50:00,2025-08-29 08:01:25,-11.416667,


nombreCompleto                   PuestoIntegracion_trabajador  \
                                                                                
0   Alejandra Gajardo Carrasco                       Asistente De Laboratorio   
1        Alejandro Ayala Salas  Operario Almacenamiento y Gestión de Residuos   
2     Alexander Araos Deramond                                       Operario   
3    Alfonso Velásquez Vergara                             Ayudante De Bodega   
4       Andres Hurtado Estrada                                       Operario   
..                         ...                                            ...   
86         Roger Diaz Jorquera                                       Operario   
87     Sebastián Miranda Rojas                           Inspector De Proceso   
88           Sergio Aros Lobos                                       Operario   
89          Víctor Ortiz López                                       Operario   
90    Wladimir Órdenes Morales                                       Operario   

   minutos_antes       fecha              
           count         min         max  
0              1  2025-08-20  2025-08-20  
1              4  2025-08-08  2025-09-03  
2              1  2025-08-13  2025-08-13  
3              1  2025-08-06  2025-08-06  
4              2  2025-08-25  2025-09-01  
..           ...         ...         ...  
86             2  2025-08-25  2025-08-28  
87             1  2025-08-06  2025-08-06  
88             1  2025-08-18  2025-08-18  
89             1  2025-08-29  2025-08-29  
90             1  2025-08-18  2025-08-18  

[91 rows x 5 columns]